<a href="https://colab.research.google.com/github/saigo24/Dimplomado_Ciencia_Datos/blob/main/tarea6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time

from IPython import display
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import transforms
from torchvision.utils import save_image
import numpy as np



Redes Generativas Adversarias

In [14]:
# Definir el generador
class Generator(nn.Module):
    def __init__(self, latent_dim, img_shape):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, img_shape),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), -1)
        return img

# Definir el discriminador
class Discriminator(nn.Module):
    def __init__(self, img_shape):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(img_shape, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)
        return validity

# Configuración
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
latent_dim = 100
img_shape = 28 * 28

# Crear generador y discriminador
generator = Generator(latent_dim, img_shape).to(device)
discriminator = Discriminator(img_shape).to(device)

# Función de pérdida y optimizadores
adversarial_loss = nn.BCELoss()
generator_optimizer = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
discriminator_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Cargar datos de entrenamiento (por ejemplo, MNIST)
dataset = MNIST(root='data', train=True, download=True, transform=transforms.ToTensor())
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# Entrenamiento
num_epochs = 10
for epoch in range(num_epochs):
    for i, (imgs, _) in enumerate(dataloader):
        batch_size = imgs.shape[0]

        # Preparar datos reales y datos de ruido
        real_imgs = imgs.view(batch_size, -1).to(device)
        z = torch.randn(batch_size, latent_dim).to(device)

        # Entrenar el discriminador
        discriminator_optimizer.zero_grad()

        real_labels = torch.ones(batch_size, 1).to(device)
        real_output = discriminator(real_imgs)
        real_loss = adversarial_loss(real_output, real_labels)

        fake_imgs = generator(z)
        fake_labels = torch.zeros(batch_size, 1).to(device)
        fake_output = discriminator(fake_imgs.detach())
        fake_loss = adversarial_loss(fake_output, fake_labels)

        discriminator_loss = (real_loss + fake_loss) / 2
        discriminator_loss.backward()
        discriminator_optimizer.step()

        # Entrenar el generador
        generator_optimizer.zero_grad()

        fake_output = discriminator(fake_imgs)
        generator_loss = adversarial_loss(fake_output, real_labels)
        generator_loss.backward()
        generator_optimizer.step()

        # Mostrar progreso del entrenamiento
        if (i+1) % 200 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(dataloader)}], "
                  f"Discriminator Loss: {discriminator_loss.item():.4f}, "
                  f"Generator Loss: {generator_loss.item():.4f}")

    # Guardar imágenes generadas durante el entrenamiento
    if (epoch+1) % 10 == 0:
        with torch.no_grad():
            z = torch.randn(64, latent_dim).to(device)
            generated_imgs = generator(z).reshape(-1, 1, 28, 28)
            save_image(generated_imgs, f"generated_images_epoch{epoch+1}.png", nrow=8, normalize=True)

Epoch [1/10], Step [200/938], Discriminator Loss: 0.5444, Generator Loss: 1.7075
Epoch [1/10], Step [400/938], Discriminator Loss: 0.5134, Generator Loss: 2.0192
Epoch [1/10], Step [600/938], Discriminator Loss: 0.4230, Generator Loss: 3.3124
Epoch [1/10], Step [800/938], Discriminator Loss: 0.4837, Generator Loss: 4.4507
Epoch [2/10], Step [200/938], Discriminator Loss: 0.4607, Generator Loss: 2.6317
Epoch [2/10], Step [400/938], Discriminator Loss: 0.4240, Generator Loss: 3.1243
Epoch [2/10], Step [600/938], Discriminator Loss: 0.4681, Generator Loss: 2.9704
Epoch [2/10], Step [800/938], Discriminator Loss: 0.4054, Generator Loss: 3.1892
Epoch [3/10], Step [200/938], Discriminator Loss: 0.4419, Generator Loss: 2.2824
Epoch [3/10], Step [400/938], Discriminator Loss: 0.4315, Generator Loss: 2.6357
Epoch [3/10], Step [600/938], Discriminator Loss: 0.4512, Generator Loss: 2.3430
Epoch [3/10], Step [800/938], Discriminator Loss: 0.4169, Generator Loss: 2.2408
Epoch [4/10], Step [200/938]

Generación de Rostros Artificiales usando DCGAN's

In [16]:
import os
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

%matplotlib inline

# Se usa una semilla para reproductibilidad
manualSeed = 999
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

Random Seed:  999


In [17]:
# Directorio del Dataset
dataroot = "/mnt/storage/Datasets/"

# Número de workers en el dataloader
workers = 2

# tamaño del lote
batch_size = 128

# tamaño base de las imágenes a la entrada
image_size = 64

# número de canales de entrenamiento
nc = 3

# tamaño delv ector latente
nz = 100

# tamaño de los mapas de características en el generador
ngf = 64

# tamaño de los mapas de características en el discriminador
ndf = 64

# Números de epocas
num_epochs = 10

# rata de aprendizaje
lr = 0.0002

# hiperparámetro Beta1 para el optimizador
beta1 = 0.5

# Número de GPUs, si no hay poner 0
ngpu = 1

In [19]:
transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ])

In [21]:
# Creamos el dataset
dataset = datasets.ImageFolder(root=dataroot+'celeba',
                           transform=transform)
# Creamos el dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)

# Decidimos en dónde se hará el entrenamiento
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

# graficamos algunas de sus imágenes
real_batch = next(iter(dataloader))
plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))

FileNotFoundError: ignored

In [ ]:
# inicialización de pesos para el discriminador y el generador
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [ ]:
# Generator Code

class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # Se entra el vector de ruido a la primera convolución transpuesta
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # tamaño actual. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # tamaño actual. (ngf*4) x 8 x 8
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # tamaño actual. (ngf*2) x 16 x 16
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # tamaño actual. (ngf) x 32 x 32
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # tamaño actual. (nc) x 64 x 64
        )

    def forward(self, input):
        return self.main(input)

In [22]:
# creamos el generador
netG = Generator(ngpu).to(device)

# para uso de múltiples GPUs
if (device.type == 'cuda') and (ngpu > 1):
    netG = nn.DataParallel(netG, list(range(ngpu)))

# aplicamos la aleatorización de pesos incializados
netG.apply(weights_init)

print(netG)

TypeError: ignored

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # entrada es una imagen (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # tamaño actual. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # tamaño actual. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # tamaño actual. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # tamaño actual. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
# Creamos el discriminador
netD = Discriminator(ngpu).to(device)

# para uso de múltiples GPUs
if (device.type == 'cuda') and (ngpu > 1):
    netD = nn.DataParallel(netD, list(range(ngpu)))

# aplicamos la aleatorización de pesos incializados
netD.apply(weights_init)

print(netD)

In [ ]:
# perdida de entropía cruzada binaria
criterion = nn.BCELoss()

# hacemos un lote de vectores aleatorios para visualizar
#  el proceso de mejora del generador
fixed_noise = torch.randn(64, nz, 1, 1, device=device)

#definimos las etiquetas de "falso" y "verdadero"
real_label = 1.
fake_label = 0.

# definimos los optimizadores de G y D
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

In [ ]:
# Training Loop

# listas para seguir el progreso
img_list = []
G_losses = []
D_losses = []
iters = 0

print("Starting Training Loop...")
# para cada epoch
for epoch in range(num_epochs):
    # Para cada lote en el epoch
    for i, data in enumerate(dataloader, 0):

        ############################
        # (1) Actualizamos la red D
        ###########################
        ## entrenamiento con las imágenes reales
        netD.zero_grad()
        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), real_label, dtype=torch.float, device=device)
        # pasamos el lote de imágenes reales al discriminador
        output = netD(real_cpu).view(-1)
        # perdida del lote real
        errD_real = criterion(output, label)
        # backpropagation
        errD_real.backward()
        D_x = output.mean().item()

        ## entrenamiento con las imágenes falsas
        # Generamos lote de vectores aleatorios
        noise = torch.randn(b_size, nz, 1, 1, device=device)
        # Generamos imágenes falsas con G
        fake = netG(noise)
        label.fill_(fake_label)
        # Aplicamos clasificación de estos lotes falsos
        output = netD(fake.detach()).view(-1)
        # perdida del lote falso
        errD_fake = criterion(output, label)
        # calculamos los gradientes de este lote, acumulado con el anterior
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        # computamos la perdida total de D como la suma de perdidas
        errD = errD_real + errD_fake
        # Update D
        optimizerD.step()

        ############################
        # (2) Actualizamos la red G
        ###########################
        netG.zero_grad()
        label.fill_(real_label)
        # al haber actualizado D, se hace otro paso con datos falsos usando el discriminador
        output = netD(fake).view(-1)
        # Calculamos la perdida del generador con base en que tan buena fue su capacidad de burlar a D
        errG = criterion(output, label)
        # Calculamos Gradientes
        errG.backward()
        D_G_z2 = output.mean().item()
        # actualizamos G
        optimizerG.step()

        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        # hacemos unas generaciones parciales para ver el desempeño de G
        if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

        iters += 1

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
real_batch = next(iter(dataloader))

# graficamos imágenes reales
plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.axis("off")
plt.title("Real Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=5, normalize=True).cpu(),(1,2,0)))

# graficamos las imágenes falsas del último epoch
plt.subplot(1,2,2)
plt.axis("off")
plt.title("Fake Images")
plt.imshow(np.transpose(img_list[-1],(1,2,0)))
plt.show()